<a href="https://colab.research.google.com/github/tathycosta/SQL_consultas/blob/main/Tarefas_SQL_Consultas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install mysql-connector-python

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.0/34.0 MB 10.8 MB/s eta 0:00:00


In [ ]:
import mysql.connector
import pandas as pd

In [ ]:
host= '34.173.32.71'
user= 'root'
password= ''
database= 'Locadora'

In [ ]:
!curl ipecho.net/plain

34.145.122.194

In [ ]:
def consulta(query, tabela):
  connection = mysql.connector.connect(
      host = host,
      user = user,
      password = password,
      database = database
  )
  cursor = connection.cursor()
  try:
    cursor.execute(query)
    result = cursor.fetchall()
    globals()[tabela] = pd.DataFrame(result, columns=cursor.column_names)
    display(globals()[tabela])
  finally:
    cursor.close()
    connection.close()

In [ ]:
consulta('''
SHOW TABLES;
''','')

,Tables_in_Locadora
0,aluguel
1,carro
2,cliente
3,marca


1 - Selecione todos os dados da tabela cliente


In [ ]:
consulta('''
DESCRIBE cliente;
''','')

,Field,Type,Null,Key,Default,Extra
0,codcliente,int,NO,PRI,None,
1,nome,varchar(50),YES,,None,
2,cidade,varchar(50),YES,,None,
3,sexo,char(1),YES,,None,
4,estado,char(2),YES,,None,
5,estadocivil,char(1),YES,,None,


2 - Liste todos os clientes que moram na cidade de "São Paulo".


In [ ]:
consulta('''
SELECT nome,cidade FROM cliente  WHERE cidade='São Paulo';
''','cidade')

,nome,cidade
0,Lúcia Andrade,São Paulo


3 - Liste os estados diferentes presentes na tabela cliente.

In [ ]:
consulta(
'''
SELECT DISTINCT estado FROM cliente;
''','Estados'
)

,estado
0,RJ
1,SP


4 - Liste todos os clientes ordenados pelo nome em ordem alfabética.


In [ ]:
consulta('''
SELECT nome FROM cliente
ORDER BY nome ASC
''','nome_asc')

,nome
0,Ana Silva
1,Bruna Pereira
2,Fernando Souza
3,Lúcia Andrade
4,Túlio Nascimento


5 - Encontre todos os clientes que são do sexo feminino e solteiros.


In [ ]:
consulta('''
SELECT nome
FROM cliente
WHERE sexo = 'F' AND estadocivil = 'S';
''','sexo_estadocivil')

,nome


6 - Liste todos os clientes cujo nome começa com a letra 'A'.

In [ ]:
consulta('''
SELECT nome
FROM cliente
WHERE nome LIKE 'A%';
''','nome_A')

,nome
0,Ana Silva


7 - Selecione os 3 primeiros registros da tabela cliente.


In [ ]:
consulta('''
SELECT nome FROM cliente LIMIT 3;
''','primeiros_3')

,nome
0,Ana Silva
1,Bruna Pereira
2,Túlio Nascimento


8 - Liste todos os carros que são da marca 'Ford' ou 'Fiat'.


In [ ]:
consulta('''
SELECT c.modelo, m.marca FROM carro AS c
INNER JOIN marca as m
ON c.codmarca=m.codmarca
WHERE m.marca='Ford' OR m.marca='Fiat';
''','inner_join')

,modelo,marca
0,Ka,Ford
1,Argo,Fiat



9 - Encontre todos os carros cujo valor está entre 100 e 150.


In [ ]:
consulta('''
SELECT *
FROM carro
WHERE valor BETWEEN 100 AND 150;
''','valor_100_150')

,codcarro,codmarca,modelo,valor
0,1,1,Ka,100.0
1,2,2,Argo,150.0
2,4,4,Polo,150.0
3,5,5,Kwid,120.0


10 - Conte quantos aluguéis foram realizados.

In [ ]:
consulta('''
SELECT COUNT(*) AS total_alugueis
FROM aluguel;
''','contagem_aluguel')

,total_alugueis
0,10


11 - Calcule o valor total de todos os carros disponíveis para aluguel.


In [ ]:
consulta('''
SELECT SUM(valor) AS valor_total
FROM carro;
''','valor_total')

,valor_total
0,690.0


12 - Calcule o valor médio dos carros.


In [ ]:
consulta('''
SELECT AVG(valor) as valor_medio FROM carro;
''','media_valores')

,valor_medio
0,138.0


13 - Encontre o menor e o maior valor de aluguel de carros.


In [ ]:
consulta('''
SELECT MIN(valor) as valor_MIN FROM carro;
''','valor_MIN')

,valor_MIN
0,100.0


In [ ]:
consulta('''
SELECT MAX(valor) as valor_max FROM carro;
''','valor_max')

,valor_max
0,170.0


14 - Agrupe os clientes por estado e conte quantos clientes existem em cada estado.

In [ ]:
consulta(
    '''
    SELECT estado, COUNT(*) AS total_clientes
    FROM cliente
    GROUP BY estado;
    ''','total_clientes_por_estado'
)

,estado,total_clientes
0,RJ,3
1,SP,2


15 - Agrupe os aluguéis por cliente e filtre apenas aqueles que fizeram mais de 2 aluguéis.


In [ ]:
consulta(
    '''
    SELECT codcliente, COUNT(*) AS total_alugueis
    FROM aluguel
    GROUP BY codcliente
    HAVING COUNT(*) > 2;
    ''','aluguel_por_cliente'
)

,codcliente,total_alugueis
0,1,3
1,2,3


16 - Liste todos os aluguéis, mostrando o nome do cliente e o modelo do carro alugado.


In [ ]:
consulta(
    '''
    SELECT a.codaluguel, c.nome AS nome_cliente, ca.modelo AS modelo_carro
    FROM aluguel a
    JOIN cliente c ON a.codcliente = c.codcliente
    JOIN carro ca ON a.codcarro = ca.codcarro;
    ''','aluguel_cliente_carro'
)

,codaluguel,nome_cliente,modelo_carro
0,1,Túlio Nascimento,Argo
1,2,Bruna Pereira,Ka
2,3,Bruna Pereira,Ka
3,4,Bruna Pereira,Onix
4,5,Ana Silva,Polo
5,6,Ana Silva,Polo
6,7,Ana Silva,Ka
7,8,Lúcia Andrade,Argo
8,9,Lúcia Andrade,Argo
9,10,Túlio Nascimento,Ka


17 - Liste todos os carros e, se houver, as informações sobre os aluguéis realizados desses carros.

In [ ]:
consulta(
    '''
    SELECT ca.codcarro, ca.modelo, a.codaluguel, a.codcliente, a.data_aluguel
    FROM carro ca
    LEFT JOIN aluguel a ON ca.codcarro = a.codcarro;
    ''','carro_aluguel'
)

,codcarro,modelo,codaluguel,codcliente,data_aluguel
0,1,Ka,2.0,2.0,2023-04-02
1,1,Ka,3.0,2.0,2023-04-03
2,1,Ka,7.0,1.0,2023-04-15
3,1,Ka,10.0,3.0,2023-04-25
4,2,Argo,1.0,3.0,2023-04-01
5,2,Argo,8.0,5.0,2023-04-19
6,2,Argo,9.0,5.0,2023-04-21
7,3,Onix,4.0,2.0,2023-04-04
8,4,Polo,5.0,1.0,2023-04-05
9,4,Polo,6.0,1.0,2023-04-13


18 - Liste todos os aluguéis e as informações sobre os carros alugados, mesmo que o carro não esteja mais no banco de dados.


In [ ]:
consulta(
    '''
    SELECT a.codaluguel, a.codcliente, a.data_aluguel, ca.codcarro, ca.modelo, ca.valor
    FROM aluguel a
    LEFT JOIN carro ca ON a.codcarro = ca.codcarro;
    ''','aluguel_carro'
)

,codaluguel,codcliente,data_aluguel,codcarro,modelo,valor
0,1,3,2023-04-01,2,Argo,150.0
1,2,2,2023-04-02,1,Ka,100.0
2,3,2,2023-04-03,1,Ka,100.0
3,4,2,2023-04-04,3,Onix,170.0
4,5,1,2023-04-05,4,Polo,150.0
5,6,1,2023-04-13,4,Polo,150.0
6,7,1,2023-04-15,1,Ka,100.0
7,8,5,2023-04-19,2,Argo,150.0
8,9,5,2023-04-21,2,Argo,150.0
9,10,3,2023-04-25,1,Ka,100.0


19 - Liste todos os clientes e os carros que alugaram, incluindo clientes que não alugaram e carros que não foram alugados.


In [ ]:
consulta(
    '''
    SELECT c.codcliente, c.nome AS nome_cliente, ca.codcarro, ca.modelo AS modelo_carro
    FROM cliente c
    LEFT JOIN aluguel a ON c.codcliente = a.codcliente
    LEFT JOIN carro ca ON a.codcarro = ca.codcarro

    UNION

    SELECT c.codcliente, c.nome AS nome_cliente, ca.codcarro, ca.modelo AS modelo_carro
    FROM carro ca
    LEFT JOIN aluguel a ON ca.codcarro = a.codcarro
    LEFT JOIN cliente c ON a.codcliente = c.codcliente;
    ''','cliente_carro'
)

,codcliente,nome_cliente,codcarro,modelo_carro
0,1.0,Ana Silva,4.0,Polo
1,1.0,Ana Silva,1.0,Ka
2,2.0,Bruna Pereira,1.0,Ka
3,2.0,Bruna Pereira,3.0,Onix
4,3.0,Túlio Nascimento,2.0,Argo
5,3.0,Túlio Nascimento,1.0,Ka
6,4.0,Fernando Souza,NaN,None
7,5.0,Lúcia Andrade,2.0,Argo
8,NaN,None,5.0,Kwid


20 - Combine os resultados de duas consultas: uma que lista clientes solteiros e outra que lista clientes casados.


In [ ]:
consulta(
    '''
    SELECT codcliente, nome, estadocivil
    FROM cliente
    WHERE estadocivil = 'S'

    UNION

    SELECT codcliente, nome, estadocivil
    FROM cliente
    WHERE estadocivil = 'C';
    ''','estadocivil'
)

,codcliente,nome,estadocivil
0,3,Túlio Nascimento,S
1,4,Fernando Souza,S
2,1,Ana Silva,C
3,2,Bruna Pereira,C
4,5,Lúcia Andrade,C


21 - Encontre os clientes que alugaram o carro com modelo 'Onix'.


In [ ]:
consulta(
    '''
    SELECT c.codcliente, c.nome
    FROM cliente c
    JOIN aluguel a ON c.codcliente = a.codcliente
    JOIN carro ca ON a.codcarro = ca.codcarro
    WHERE ca.modelo = 'Onix';
    ''','cliente_carro_onix'
)

,codcliente,nome
0,2,Bruna Pereira


22 - Liste os modelos de carros e o número de vezes que cada um foi alugado.

In [ ]:
consulta(
    '''
    SELECT ca.modelo, COUNT(a.codaluguel) AS total_alugueis
    FROM carro ca
    LEFT JOIN aluguel a ON ca.codcarro = a.codcarro
    GROUP BY ca.modelo;
    ''','carro_alugueis'
)

,modelo,total_alugueis
0,Ka,4
1,Argo,3
2,Onix,1
3,Polo,2
4,Kwid,0


23 - Liste o nome dos clientes e o número de aluguéis que cada um fez.


In [ ]:
consulta(
    '''
    SELECT c.nome, COUNT(a.codaluguel) AS total_alugueis
    FROM cliente c
    LEFT JOIN aluguel a ON c.codcliente = a.codcliente
    GROUP BY c.nome;
    ''','cliente_alugueis'
)

,nome,total_alugueis
0,Ana Silva,3
1,Bruna Pereira,3
2,Túlio Nascimento,2
3,Fernando Souza,0
4,Lúcia Andrade,2


24 - Crie uma consulta que retorne o nome dos clientes e uma coluna adicional chamada status_civil que classifica os clientes como "Solteiro" ou "Casado".


In [ ]:
consulta(
    '''
    SELECT nome,
       CASE
           WHEN estadocivil = 'S' THEN 'Solteiro'
           WHEN estadocivil = 'C' THEN 'Casado'
           ELSE 'Outro'
       END AS status_civil
    FROM cliente;
    ''','status_civil'
)

,nome,status_civil
0,Ana Silva,Casado
1,Bruna Pereira,Casado
2,Túlio Nascimento,Solteiro
3,Fernando Souza,Solteiro
4,Lúcia Andrade,Casado


25 - Liste os nomes dos clientes e as datas em que alugaram carros.


In [ ]:
consulta(
    '''
    SELECT c.nome, a.data_aluguel
    FROM cliente c
    JOIN aluguel a ON c.codcliente = a.codcliente
    ORDER BY c.nome, a.data_aluguel;
    ''','cliente_data_aluguel'
)

,nome,data_aluguel
0,Ana Silva,2023-04-05
1,Ana Silva,2023-04-13
2,Ana Silva,2023-04-15
3,Bruna Pereira,2023-04-02
4,Bruna Pereira,2023-04-03
5,Bruna Pereira,2023-04-04
6,Lúcia Andrade,2023-04-19
7,Lúcia Andrade,2023-04-21
8,Túlio Nascimento,2023-04-01
9,Túlio Nascimento,2023-04-25


26 - Liste todos os modelos de carros e suas respectivas marcas.


In [ ]:
consulta(
    '''
    SELECT ca.modelo, m.marca
    FROM carro ca
    LEFT JOIN marca m ON ca.codmarca = m.codmarca;
    ''','modelo_marca'
)

,modelo,marca
0,Ka,Ford
1,Argo,Fiat
2,Onix,Chevrolet
3,Polo,Volkswagen
4,Kwid,Renault


27 - Liste todos os aluguéis e os nomes dos clientes que os realizaram.


In [ ]:
consulta(
    '''
    SELECT a.codaluguel, c.nome AS nome_cliente
    FROM aluguel a
    JOIN cliente c ON a.codcliente = c.codcliente
    ORDER BY a.codaluguel;
    ''','aluguel_cliente'
)

,codaluguel,nome_cliente
0,1,Túlio Nascimento
1,2,Bruna Pereira
2,3,Bruna Pereira
3,4,Bruna Pereira
4,5,Ana Silva
5,6,Ana Silva
6,7,Ana Silva
7,8,Lúcia Andrade
8,9,Lúcia Andrade
9,10,Túlio Nascimento


28 - Encontre o nome dos clientes e os modelos de carros que alugaram.


In [ ]:
consulta(
    '''
    SELECT c.nome, ca.modelo
    FROM cliente c
    JOIN aluguel a ON c.codcliente = a.codcliente
    JOIN carro ca ON a.codcarro = ca.codcarro;
    ''','cliente_carro'
)

,nome,modelo
0,Túlio Nascimento,Argo
1,Bruna Pereira,Ka
2,Bruna Pereira,Ka
3,Bruna Pereira,Onix
4,Ana Silva,Polo
5,Ana Silva,Polo
6,Ana Silva,Ka
7,Lúcia Andrade,Argo
8,Lúcia Andrade,Argo
9,Túlio Nascimento,Ka


29 - Liste todos os carros e, se houver, mostre as datas em que foram alugados.


In [ ]:
consulta(
    '''
    SELECT ca.codcarro, ca.modelo, a.data_aluguel
    FROM carro ca
    LEFT JOIN aluguel a ON ca.codcarro = a.codcarro;
    ''','carro_aluguel'
)

,codcarro,modelo,data_aluguel
0,1,Ka,2023-04-02
1,1,Ka,2023-04-03
2,1,Ka,2023-04-15
3,1,Ka,2023-04-25
4,2,Argo,2023-04-01
5,2,Argo,2023-04-19
6,2,Argo,2023-04-21
7,3,Onix,2023-04-04
8,4,Polo,2023-04-05
9,4,Polo,2023-04-13


30 - Encontre os clientes que moram na mesma cidade e estado e que alugaram carros.


In [ ]:
consulta(
    '''
    SELECT c1.nome AS cliente1, c2.nome AS
    cliente2, c1.cidade, c1.estado
    FROM cliente c1
    JOIN cliente c2 ON c1.codcliente <> c2.codcliente
    AND c1.cidade = c2.cidade
    AND c1.estado = c2.estado;
    ''','cidade_estado'
)

,cliente1,cliente2,cidade,estado
0,Túlio Nascimento,Ana Silva,Duque de Caxias,RJ
1,Ana Silva,Túlio Nascimento,Duque de Caxias,RJ


 31 - Liste os nomes dos clientes e os carros que eles alugaram, incluindo os clientes que não alugaram nenhum carro.


In [ ]:
consulta(
    '''
    SELECT c.nome, ca.modelo
    FROM cliente c
    LEFT JOIN aluguel a ON c.codcliente = a.codcliente
    LEFT JOIN carro ca ON a.codcarro = ca.codcarro
    ORDER BY c.nome;
    ''','cliente_carro'
)

,nome,modelo
0,Ana Silva,Polo
1,Ana Silva,Polo
2,Ana Silva,Ka
3,Bruna Pereira,Ka
4,Bruna Pereira,Ka
5,Bruna Pereira,Onix
6,Fernando Souza,None
7,Lúcia Andrade,Argo
8,Lúcia Andrade,Argo
9,Túlio Nascimento,Argo


32 - Liste todas as marcas e, se houver, os modelos de carros associados.


In [ ]:
consulta(
    '''
    SELECT m.marca, ca.modelo
    FROM marca m
    LEFT JOIN carro ca ON m.codmarca = ca.codmarca;
    ''','marca_carro'
)

,marca,modelo
0,Ford,Ka
1,Fiat,Argo
2,Chevrolet,Onix
3,Volkswagen,Polo
4,Renault,Kwid


33 - Encontre todos os aluguéis e o estado civil dos clientes que alugaram os carros.

In [ ]:
consulta(
    '''
    SELECT a.codaluguel, c.estadocivil
    FROM aluguel a
    JOIN cliente c ON a.codcliente = c.codcliente;
    ''','aluguel_estadocivil'
)

,codaluguel,estadocivil
0,5,C
1,6,C
2,7,C
3,2,C
4,3,C
5,4,C
6,1,S
7,10,S
8,8,C
9,9,C


34 - Liste os clientes e as cidades em que moram para aqueles que alugaram carros.


In [ ]:
consulta(
    '''
    SELECT c.nome, c.cidade
    FROM cliente c
    JOIN aluguel a ON c.codcliente = a.codcliente;
    ''','cliente_cidade'
)

,nome,cidade
0,Ana Silva,Duque de Caxias
1,Ana Silva,Duque de Caxias
2,Ana Silva,Duque de Caxias
3,Bruna Pereira,Niterói
4,Bruna Pereira,Niterói
5,Bruna Pereira,Niterói
6,Túlio Nascimento,Duque de Caxias
7,Túlio Nascimento,Duque de Caxias
8,Lúcia Andrade,São Paulo
9,Lúcia Andrade,São Paulo


35 - Liste os clientes que alugaram carros e os modelos de carros que eles alugaram, incluindo a marca do carro.


In [ ]:
consulta(
    '''
    SELECT c.nome, ca.modelo, m.marca
    FROM cliente c
    JOIN aluguel a ON c.codcliente = a.codcliente
    JOIN carro ca ON a.codcarro = ca.codcarro
    JOIN marca m ON ca.codmarca
    = m.codmarca;
    ''','cliente_carro_marca'
)

,nome,modelo,marca
0,Bruna Pereira,Ka,Ford
1,Bruna Pereira,Ka,Ford
2,Ana Silva,Ka,Ford
3,Túlio Nascimento,Ka,Ford
4,Túlio Nascimento,Argo,Fiat
5,Lúcia Andrade,Argo,Fiat
6,Lúcia Andrade,Argo,Fiat
7,Bruna Pereira,Onix,Chevrolet
8,Ana Silva,Polo,Volkswagen
9,Ana Silva,Polo,Volkswagen


36 - Encontre todos os aluguéis realizados em uma cidade específica e liste os nomes dos clientes e os modelos de carros alugados.


In [ ]:
consulta(
    '''
    SELECT a.codaluguel, c.nome, ca.modelo
    FROM aluguel a
    JOIN cliente c ON a.codcliente = c.codcliente
    JOIN carro ca ON a.codcarro = ca.codcarro
    WHERE c.cidade = 'São Paulo';
    ''','cidade_cliente_carro'
)

,codaluguel,nome,modelo
0,8,Lúcia Andrade,Argo
1,9,Lúcia Andrade,Argo


37 - Liste os modelos de carros e a quantidade de vezes que foram alugados.

In [ ]:
consulta(
    '''
    SELECT ca.modelo, COUNT(a.codaluguel) AS total_alugueis
    FROM carro ca
    LEFT JOIN aluguel a ON ca.codcarro = a.codcarro
    GROUP BY ca.modelo;
    ''','carro_alugueis'
)

,modelo,total_alugueis
0,Ka,4
1,Argo,3
2,Onix,1
3,Polo,2
4,Kwid,0


38 - Encontre os clientes que alugaram mais de um carro.


In [ ]:
consulta(
    '''
    SELECT c.codcliente, c.nome, COUNT(a.codcarro) AS total_carros_alugados
    FROM cliente c
    JOIN aluguel a ON c.codcliente = a.codcliente
    GROUP BY c.codcliente, c.nome
    HAVING COUNT(a.codcarro) > 1;
    ''','cliente_carro_alugados'
)

,codcliente,nome,total_carros_alugados
0,3,Túlio Nascimento,2
1,2,Bruna Pereira,3
2,1,Ana Silva,3
3,5,Lúcia Andrade,2


39 - Liste os clientes, as datas de aluguel e o valor do carro alugado.


In [ ]:
consulta(
    '''
    SELECT c.nome, a.data_aluguel, ca.valor
    FROM cliente c
    JOIN aluguel a ON c.codcliente = a.codcliente
    JOIN carro ca ON a.codcarro = ca.codcarro;
    ''','cliente_data_carro'
)

,nome,data_aluguel,valor
0,Túlio Nascimento,2023-04-01,150.0
1,Bruna Pereira,2023-04-02,100.0
2,Bruna Pereira,2023-04-03,100.0
3,Bruna Pereira,2023-04-04,170.0
4,Ana Silva,2023-04-05,150.0
5,Ana Silva,2023-04-13,150.0
6,Ana Silva,2023-04-15,100.0
7,Lúcia Andrade,2023-04-19,150.0
8,Lúcia Andrade,2023-04-21,150.0
9,Túlio Nascimento,2023-04-25,100.0


40 - Liste os modelos de carros que nunca foram alugados.


In [ ]:
consulta(
    '''
    SELECT ca.modelo
    FROM carro ca
    LEFT JOIN aluguel a ON ca.codcarro = a.codcarro
    WHERE a.codcarro IS NULL;
    ''','carro_nao_alugado'
)

,modelo
0,Kwid


41 - Encontre os clientes que alugaram carros e o total gasto em aluguéis.


In [ ]:
consulta(
    '''
    SELECT c.nome, SUM(ca.valor) AS total_gasto
    FROM cliente c
    JOIN aluguel a ON c.codcliente = a.codcliente
    JOIN carro ca ON a.codcarro = ca.codcarro
    GROUP BY c.nome;
    ''','cliente_total_gasto'
)

,nome,total_gasto
0,Túlio Nascimento,250.0
1,Bruna Pereira,370.0
2,Ana Silva,400.0
3,Lúcia Andrade,300.0


42 - Liste as marcas de carros e o número total de aluguéis por marca.


In [ ]:
consulta(
    '''
    SELECT m.marca, COUNT(a.codaluguel) AS total_alugueis
    FROM marca m
    LEFT JOIN carro ca ON m.codmarca = ca.codmarca
    LEFT JOIN aluguel a ON ca.codcarro = a.codcarro
    GROUP BY m.marca;
    ''','marca_alugueis'
)

,marca,total_alugueis
0,Ford,4
1,Fiat,3
2,Chevrolet,1
3,Volkswagen,2
4,Renault,0


43 - Encontre os clientes que alugaram carros em um determinado mês e ano, listando os modelos de carros e as datas de aluguel.


In [ ]:
consulta(
    '''
    SELECT c.nome AS nome_cliente, ca.modelo AS modelo_carro, a.data_aluguel
    FROM cliente c
    JOIN aluguel a ON c.codcliente = a.codcliente
    JOIN carro ca ON a.codcarro = ca.codcarro
    WHERE YEAR(a.data_aluguel) = 2023 AND MONTH(a.data_aluguel) = 4
    ORDER BY a.data_aluguel;
    ''','cliente_carro_data'
)

,nome_cliente,modelo_carro,data_aluguel
0,Túlio Nascimento,Argo,2023-04-01
1,Bruna Pereira,Ka,2023-04-02
2,Bruna Pereira,Ka,2023-04-03
3,Bruna Pereira,Onix,2023-04-04
4,Ana Silva,Polo,2023-04-05
5,Ana Silva,Polo,2023-04-13
6,Ana Silva,Ka,2023-04-15
7,Lúcia Andrade,Argo,2023-04-19
8,Lúcia Andrade,Argo,2023-04-21
9,Túlio Nascimento,Ka,2023-04-25


44 - Liste os clientes que moram em estados diferentes e que alugaram carros, junto com os modelos alugados.


In [ ]:
consulta(
    '''
    SELECT DISTINCT c.nome AS nome_cliente, c.estado, ca.modelo AS modelo_carro
    FROM cliente c
    JOIN aluguel a ON c.codcliente = a.codcliente
    JOIN carro ca ON a.codcarro = ca.codcarro
    WHERE c.estado IN (
      SELECT estado
      FROM cliente
      GROUP BY estado
      HAVING COUNT(DISTINCT codcliente) > 1
)
    ORDER BY c.estado, c.nome;
    ''','cliente_carro_estado'
)

,nome_cliente,estado,modelo_carro
0,Ana Silva,RJ,Polo
1,Ana Silva,RJ,Ka
2,Bruna Pereira,RJ,Ka
3,Bruna Pereira,RJ,Onix
4,Túlio Nascimento,RJ,Argo
5,Túlio Nascimento,RJ,Ka
6,Lúcia Andrade,SP,Argo


45 - Encontre os clientes que alugaram carros, os modelos de carros alugados, as marcas desses carros e as datas de aluguel, ordenando pelos nomes dos clientes.


In [ ]:
consulta(
    '''
    SELECT c.nome AS nome_cliente, ca.modelo AS modelo_carro, m.marca, a.data_aluguel
    FROM cliente c
    JOIN aluguel a ON c.codcliente = a.codcliente
    JOIN carro ca ON a.codcarro = ca.codcarro
    JOIN marca m ON ca.codmarca = m.codmarca
    ORDER BY c.nome, a.data_aluguel;
    ''', 'cliente_carro_marca_data'
)

,nome_cliente,modelo_carro,marca,data_aluguel
0,Ana Silva,Polo,Volkswagen,2023-04-05
1,Ana Silva,Polo,Volkswagen,2023-04-13
2,Ana Silva,Ka,Ford,2023-04-15
3,Bruna Pereira,Ka,Ford,2023-04-02
4,Bruna Pereira,Ka,Ford,2023-04-03
5,Bruna Pereira,Onix,Chevrolet,2023-04-04
6,Lúcia Andrade,Argo,Fiat,2023-04-19
7,Lúcia Andrade,Argo,Fiat,2023-04-21
8,Túlio Nascimento,Argo,Fiat,2023-04-01
9,Túlio Nascimento,Ka,Ford,2023-04-25


46 - Liste os clientes que alugaram carros mais de uma vez e os detalhes dos carros que alugaram.


In [ ]:
consulta(
    '''
    SELECT c.nome AS nome_cliente, ca.modelo AS modelo_carro, m.marca, COUNT(a.codaluguel) AS total_alugueis
    FROM cliente c
    JOIN aluguel a ON c.codcliente = a.codcliente
    JOIN carro ca ON a.codcarro = ca.codcarro
    JOIN marca m ON ca.codmarca = m.codmarca
    GROUP BY c.codcliente, ca.codcarro
    HAVING COUNT(a.codaluguel) > 1
    ORDER BY c.nome, ca.modelo;
    ''', 'cliente_carro_marca_alugueis'
)


,nome_cliente,modelo_carro,marca,total_alugueis
0,Ana Silva,Polo,Volkswagen,2
1,Bruna Pereira,Ka,Ford,2
2,Lúcia Andrade,Argo,Fiat,2


47 - Encontre os modelos de carros que foram alugados por clientes de cidades específicas e liste as marcas desses carros.


In [ ]:
consulta(
    '''
    SELECT DISTINCT ca.modelo AS modelo_carro, m.marca
    FROM carro ca
    JOIN aluguel a ON ca.codcarro = a.codcarro
    JOIN cliente c ON a.codcliente = c.codcliente
    JOIN marca m ON ca.codmarca
    = m.codmarca
    WHERE c.cidade IN ('São Paulo', 'Rio de Janeiro')
    ORDER BY ca.modelo;
    ''', 'carro_marca_cidades'
)

,modelo_carro,marca
0,Argo,Fiat


48 - Liste os clientes que alugaram carros, os modelos e as marcas dos carros, e o total gasto em aluguéis, agrupado por cliente.


In [ ]:
consulta(
    '''
    SELECT c.nome AS nome_cliente, ca.modelo AS modelo_carro, m.marca,
    SUM(ca.valor) AS total_gasto
    FROM cliente c
    JOIN aluguel a ON c.codcliente = a.codcliente
    JOIN carro ca ON a.codcarro = ca.codcarro
    JOIN marca m ON ca.codmarca
    = m.codmarca
    GROUP BY c.codcliente, ca.codcarro
    ORDER BY c.nome, ca.modelo;
    ''', 'cliente_carro_marca_total'
)


,nome_cliente,modelo_carro,marca,total_gasto
0,Ana Silva,Ka,Ford,100.0
1,Ana Silva,Polo,Volkswagen,300.0
2,Bruna Pereira,Ka,Ford,200.0
3,Bruna Pereira,Onix,Chevrolet,170.0
4,Lúcia Andrade,Argo,Fiat,300.0
5,Túlio Nascimento,Argo,Fiat,150.0
6,Túlio Nascimento,Ka,Ford,100.0


49 - Encontre os carros que foram alugados nos últimos 30 dias e os clientes que os alugaram.


In [ ]:
consulta(
    '''
    SELECT ca.modelo AS modelo_carro, c.nome AS nome_cliente
    FROM carro ca
    JOIN aluguel a ON ca.codcarro = a.codcarro
    JOIN cliente c ON a.codcliente = c.codcliente
    WHERE a.data_aluguel >= DATE_SUB(CURDATE(), INTERVAL 30 DAY)
    ORDER BY ca.modelo, c.nome;
    ''', 'carro_cliente_ultimos_30_dias'
)

,modelo_carro,nome_cliente


50 - Liste os clientes, os carros que alugaram, as marcas desses carros, e o valor total gasto em aluguéis, filtrando por um intervalo de datas específico.


In [ ]:
consulta(
    '''
    SELECT c.nome AS nome_cliente, ca.modelo AS modelo_carro, m.marca,
    SUM(ca.valor) AS total_gasto
    FROM cliente c
    JOIN aluguel a ON c.codcliente
    = a.codcliente
    JOIN carro ca ON a.codcarro = ca.codcarro
    JOIN marca m ON ca.codmarca
    = m.codmarca
    WHERE a.data_aluguel BETWEEN '2023-04-01' AND '2023-04-30'
    GROUP BY c.codcliente, ca.codcarro
    ORDER BY c.nome, ca.modelo;
    ''', 'cliente_carro_marca_total_intervalo'
)

,nome_cliente,modelo_carro,marca,total_gasto
0,Ana Silva,Ka,Ford,100.0
1,Ana Silva,Polo,Volkswagen,300.0
2,Bruna Pereira,Ka,Ford,200.0
3,Bruna Pereira,Onix,Chevrolet,170.0
4,Lúcia Andrade,Argo,Fiat,300.0
5,Túlio Nascimento,Argo,Fiat,150.0
6,Túlio Nascimento,Ka,Ford,100.0
